In [1]:
import pandas as pd
import numpy as np
import urllib.request
from sentimentLDA import *
import os
import urllib
import tarfile
import re

import warnings
warnings.filterwarnings("ignore")

In [2]:
text_source = pd.read_csv('./Text_data/scrapped_articles_master.csv', header=[0])
cycles_ref = text_source['Cycle_month'].unique()

In [3]:
def extract_cyclemonth(df, cyclemonth):
    text_sample = df[df['Cycle_month'] == cyclemonth]['Text'].values.tolist()
    
    # Remove the leading and ending characters
    # And add \n
    pattern = re.compile(r'^[^a-zA-Z]*|[^a-zA-Z]*$')

    for i, text in enumerate(text_sample):
        text_sample[i] = "\n" + pattern.sub('', text).strip('\n') + "\n"

    return text_sample

In [4]:
def obtain_words(df, cyclemonth):
    text_sample = extract_cyclemonth(df, cyclemonth)

    sampler = SentimentLDAGibbsSampler(1, 2.5, 0.1, 0.3)
    sampler.run(text_sample, 80, f"./Sentiment_data/dll_storage/{cyclemonth}.dll", True)

    return sampler.getTopKWords(25)

In [13]:
xx = obtain_words(text_source, cycles_ref[0])

Starting iteration 1 of 80
Starting iteration 2 of 80
Starting iteration 3 of 80
Starting iteration 4 of 80
Starting iteration 5 of 80
Starting iteration 6 of 80
Starting iteration 7 of 80
Starting iteration 8 of 80
Starting iteration 9 of 80
Starting iteration 10 of 80
Starting iteration 11 of 80
Starting iteration 12 of 80
Starting iteration 13 of 80
Starting iteration 14 of 80
Starting iteration 15 of 80
Starting iteration 16 of 80
Starting iteration 17 of 80
Starting iteration 18 of 80
Starting iteration 19 of 80
Starting iteration 20 of 80
Starting iteration 21 of 80
Starting iteration 22 of 80
Starting iteration 23 of 80
Starting iteration 24 of 80
Starting iteration 25 of 80
Starting iteration 26 of 80
Starting iteration 27 of 80
Starting iteration 28 of 80
Starting iteration 29 of 80
Starting iteration 30 of 80
Starting iteration 31 of 80
Starting iteration 32 of 80
Starting iteration 33 of 80
Starting iteration 34 of 80
Starting iteration 35 of 80
Starting iteration 36 of 80
S

In [14]:
xx[1][2]

['appl',
 'compani',
 'said',
 'market',
 'samsung',
 'iphon',
 'new',
 'verizon',
 'smartphon',
 'chip',
 'year',
 'share',
 'devic',
 'wireless',
 'analyst',
 'time',
 'rim',
 'use',
 'tv',
 'like',
 'execut',
 'chief',
 'product',
 'china',
 'tablet']

In [15]:
top_sentiment_words = pd.DataFrame("Dummy", index = cycles_ref, columns = ['negative', 'positive'])

In [16]:
for idx in cycles_ref:
    print(f"Current idx: {idx}")
    obtained_words = obtain_words(text_source, idx)
    top_sentiment_words.loc[idx, 'negative'] = obtained_words[0][2]
    top_sentiment_words.loc[idx, 'positive'] = obtained_words[1][2]

Current idx: 201201
Starting iteration 1 of 80
Starting iteration 2 of 80
Starting iteration 3 of 80
Starting iteration 4 of 80
Starting iteration 5 of 80
Starting iteration 6 of 80
Starting iteration 7 of 80
Starting iteration 8 of 80
Starting iteration 9 of 80
Starting iteration 10 of 80
Starting iteration 11 of 80
Starting iteration 12 of 80
Starting iteration 13 of 80
Starting iteration 14 of 80
Starting iteration 15 of 80
Starting iteration 16 of 80
Starting iteration 17 of 80
Starting iteration 18 of 80
Starting iteration 19 of 80
Starting iteration 20 of 80
Starting iteration 21 of 80
Starting iteration 22 of 80
Starting iteration 23 of 80
Starting iteration 24 of 80
Starting iteration 25 of 80
Starting iteration 26 of 80
Starting iteration 27 of 80
Starting iteration 28 of 80
Starting iteration 29 of 80
Starting iteration 30 of 80
Starting iteration 31 of 80
Starting iteration 32 of 80
Starting iteration 33 of 80
Starting iteration 34 of 80
Starting iteration 35 of 80
Starting 

In [17]:
top_sentiment_words.to_csv("./Sentiment_data/top_sentiment_words.csv")

## Update Sentiment Words

In [5]:
original_sentiment = pd.read_csv('./Sentiment_data/top_sentiment_words.csv')

In [14]:
diff_list = np.setdiff1d(cycles_ref, original_sentiment['Unnamed: 0'].values)
new_part = text_source[text_source['Cycle_month'].isin(diff_list)]

In [16]:
new_sentiment_words = pd.DataFrame("Dummy", index = diff_list, columns = ['negative', 'positive'])
for idx in diff_list:
    print(f"Current idx: {idx}")
    obtained_words = obtain_words(new_part, idx)
    new_sentiment_words.loc[idx, 'negative'] = obtained_words[0][2]
    new_sentiment_words.loc[idx, 'positive'] = obtained_words[1][2]

Current idx: 201608
Starting iteration 1 of 80
Starting iteration 2 of 80
Starting iteration 3 of 80
Starting iteration 4 of 80
Starting iteration 5 of 80
Starting iteration 6 of 80
Starting iteration 7 of 80
Starting iteration 8 of 80
Starting iteration 9 of 80
Starting iteration 10 of 80
Starting iteration 11 of 80
Starting iteration 12 of 80
Starting iteration 13 of 80
Starting iteration 14 of 80
Starting iteration 15 of 80
Starting iteration 16 of 80
Starting iteration 17 of 80
Starting iteration 18 of 80
Starting iteration 19 of 80
Starting iteration 20 of 80
Starting iteration 21 of 80
Starting iteration 22 of 80
Starting iteration 23 of 80
Starting iteration 24 of 80
Starting iteration 25 of 80
Starting iteration 26 of 80
Starting iteration 27 of 80
Starting iteration 28 of 80
Starting iteration 29 of 80
Starting iteration 30 of 80
Starting iteration 31 of 80
Starting iteration 32 of 80
Starting iteration 33 of 80
Starting iteration 34 of 80
Starting iteration 35 of 80
Starting 

In [19]:
original_sentiment.index = original_sentiment['Unnamed: 0']
original_sentiment = original_sentiment.drop(['Unnamed: 0'], axis = 1)

In [36]:
new_sentiment_words.loc[201608, 'negative'] = str(new_sentiment_words.loc[201608, 'negative'])
new_sentiment_words.loc[201609, 'negative'] = str(new_sentiment_words.loc[201609, 'negative'])
new_sentiment_words.loc[201610, 'negative'] = str(new_sentiment_words.loc[201610, 'negative'])
new_sentiment_words.loc[201608, 'positive'] = str(new_sentiment_words.loc[201608, 'positive'])
new_sentiment_words.loc[201609, 'positive'] = str(new_sentiment_words.loc[201609, 'positive'])
new_sentiment_words.loc[201610, 'positive'] = str(new_sentiment_words.loc[201610, 'positive'])

In [37]:
final_sentiment_words = pd.concat([original_sentiment, new_sentiment_words], axis = 0).sort_index(ascending = True)

In [38]:
final_sentiment_words.to_csv("./Sentiment_data/top_sentiment_words.csv")